In [74]:
# IMPORT LIBRARIES

# This is an rnn for the lrg_brg_INSERT data from 2019-07-07

# Import data processing tools
from dataProcess import *
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib ipympl
import numpy as np
import pickle
import re

# Import ML tools
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.optimizers import Adam
from keras import regularizers
from keras.utils import np_utils

# Import tools needed for visualization
from IPython.display import Image, display

In [75]:
# READ DATA

# read the pickle file, which contains 1) file names and 2) labels
with open('2019-07-07_annot.pkl', 'rb') as f_pkl:
    data_pkl = pickle.load(f_pkl)

# lrg_brg: stores file names
# file_labels: stores labels for files
files = []
file_labels = []
for i in data_pkl:
    if(re.findall("lrg_brg_INSERT", i) != []):
        files.append(i[7:])
        if(data_pkl[i] == "FAULT" or data_pkl[i] == "FAIL"):
            file_labels.append(False)
        else:
            file_labels.append(True)

# data: stores forces/torques
# data_labels: stores labels for data
data = []
data_labels = []
for j in range(0, len(files)):
    frcSeries = timeseries_from_file(files[j],'insert_part_tilt_record::tilt_in','gripPose')
    for i in frcSeries:
        data.append(i)
        data_labels.append(file_labels[j])

# store all data in a Pandas dataframe  
columns = ['F_x', 'F_y', 'F_z', 'T_x', 'T_y', 'T_z']
data_frame = pd.DataFrame(data, columns = columns)

# store all class labels in a Pandas series
label_series = pd.Series(data_labels)

Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements
Obtained a time series with 18 elements


In [110]:
# TRAIN DATA

# features: an array of forces/torques
# categorical_labels: a list of fail[1 0] and success[0 1] labels
features = np.array(data_frame).astype(float)
features.reshape(396,1,6)

labels = label_series
labels = labels.astype(int)
categorical_labels = np_utils.to_categorical(labels, num_classes = 2).astype(int)

# call train_test_split on data
X_train, X_test, Y_train, Y_test = train_test_split(features, categorical_labels, test_size = 0.2)
X_train = X_train.reshape(X_train.shape[0],6,1)
X_test = X_test.reshape(X_test.shape[0],6,1)

HIDDEN_SIZE = 64
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
model = Sequential()
model.add(LSTM(HIDDEN_SIZE, return_sequences = False, input_shape = (6, 1), unroll=True))
model.add(Dense(HIDDEN_SIZE))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation("softmax"))
model.compile(loss='binary_crossentropy',optimizer="rmsprop")

NB_EPOCH=200
BATCH_SIZE = 8
VERBOSE = 1
VALIDATION_SPLIT = 0.2

history = model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH,verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

Train on 252 samples, validate on 64 samples
Epoch 1/200
252/252 [==============================] - 9s 35ms/step - loss: 0.6932 - val_loss: 0.7067
Epoch 2/200
252/252 [==============================] - 1s 3ms/step - loss: 0.6914 - val_loss: 0.7093
Epoch 3/200
252/252 [==============================] - 1s 4ms/step - loss: 0.6916 - val_loss: 0.7037
Epoch 4/200
252/252 [==============================] - 1s 3ms/step - loss: 0.6903 - val_loss: 0.7061
Epoch 5/200
252/252 [==============================] - 1s 4ms/step - loss: 0.6899 - val_loss: 0.7024
Epoch 6/200
252/252 [==============================] - 1s 4ms/step - loss: 0.6905 - val_loss: 0.7033
Epoch 7/200
252/252 [==============================] - 1s 3ms/step - loss: 0.6907 - val_loss: 0.7084
Epoch 8/200
252/252 [==============================] - 1s 4ms/step - loss: 0.6912 - val_loss: 0.7147
Epoch 9/200
252/252 [==============================] - 1s 4ms/step - loss: 0.6911 - val_loss: 0.7073
Epoch 10/200
252/252 [=======================

Epoch 81/200
252/252 [==============================] - 1s 3ms/step - loss: 0.6979 - val_loss: 0.7463
Epoch 82/200
252/252 [==============================] - 0s 2ms/step - loss: 0.6988 - val_loss: 0.7095
Epoch 83/200
252/252 [==============================] - 0s 2ms/step - loss: 0.6966 - val_loss: 0.6929
Epoch 84/200
252/252 [==============================] - 1s 2ms/step - loss: 0.6960 - val_loss: 0.7677
Epoch 85/200
252/252 [==============================] - 0s 2ms/step - loss: 0.7009 - val_loss: 0.7455
Epoch 86/200
252/252 [==============================] - 0s 2ms/step - loss: 0.7002 - val_loss: 0.7338
Epoch 87/200
252/252 [==============================] - 0s 2ms/step - loss: 0.6988 - val_loss: 0.7108
Epoch 88/200
252/252 [==============================] - 0s 2ms/step - loss: 0.6993 - val_loss: 0.7278
Epoch 89/200
252/252 [==============================] - 0s 2ms/step - loss: 0.6979 - val_loss: 0.6921
Epoch 90/200
252/252 [==============================] - 0s 2ms/step - loss: 0.6910

252/252 [==============================] - 1s 2ms/step - loss: 0.6925 - val_loss: 0.6977
Epoch 161/200
252/252 [==============================] - 0s 2ms/step - loss: 0.6947 - val_loss: 0.7290
Epoch 162/200
252/252 [==============================] - 0s 2ms/step - loss: 0.6927 - val_loss: 0.7185
Epoch 163/200
252/252 [==============================] - 0s 2ms/step - loss: 0.6920 - val_loss: 0.7296
Epoch 164/200
252/252 [==============================] - 0s 2ms/step - loss: 0.6963 - val_loss: 0.7041
Epoch 165/200
252/252 [==============================] - 0s 2ms/step - loss: 0.6934 - val_loss: 0.7004
Epoch 166/200
252/252 [==============================] - 0s 2ms/step - loss: 0.6932 - val_loss: 0.6950
Epoch 167/200
252/252 [==============================] - 0s 2ms/step - loss: 0.6926 - val_loss: 0.7019
Epoch 168/200
252/252 [==============================] - 1s 2ms/step - loss: 0.6936 - val_loss: 0.6935
Epoch 169/200
252/252 [==============================] - 0s 2ms/step - loss: 0.6936 - v

In [111]:
# MAKE PREDICTIONS

predictions = np.round(model.predict(X_test))
errors = abs(predictions-Y_test)
print('Error: ',sum(errors)/len(Y_test)*100,'%')

Error:  [37.5 37.5] %
